In [7]:
from datetime import datetime,timedelta,date
from time import time
import requests
import pandas as pd
from datetime import date,datetime
import json
from ta.volatility import BollingerBands
from ta import add_all_ta_features


In [8]:
date0,date1 = datetime(2016,1,1) ,datetime.now() - timedelta(days=1)

days = (date1 - date0).days
print(days)

2322


In [9]:
DataFrame = pd.DataFrame(columns = ["time","low","high","open","close","volume","date"])


In [10]:
DataFrame

,time,low,high,open,close,volume,date


In [11]:
dates = []
d0 = date0
d1 = date1
date =[]
for i in range(1,(days//300)+ 1):
    
    d1 = d0 + timedelta(days=300)
    date = [d0.strftime("%Y-%m-%d"),d1.strftime("%Y-%m-%d")]
    dates.append(date)
    d0 = d1

d1 = d0 + timedelta(days= days % 300)
date = [d0.strftime("%Y-%m-%d"),d1.strftime("%Y-%m-%d")]
dates.append(date)


In [12]:

#API_MAIN_LINK
url = "https://api.exchange.coinbase.com"

#Currency name to scrap price 
sym = 'BTC-USD'

#start & ending time of period with size of candle
for i in dates:
    parameters = {
        'start':i[0],
        'end':i[1],
        'granularity':'86400'
    }
    data = requests.get(f"{url}/products/{sym}/candles",params=parameters,headers={'Accepts':'application/json'})
    df = pd.DataFrame(data.json(),columns = ["time","low","high","open","close","volume"])
    df["date"] = pd.to_datetime(df["time"],unit='s')
    DataFrame = DataFrame.append(df)


In [13]:
DataFrame.sort_values(by='date',inplace=True)



In [14]:
DataFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2323 entries, 299 to 0
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    2323 non-null   object        
 1   low     2323 non-null   float64       
 2   high    2323 non-null   float64       
 3   open    2323 non-null   float64       
 4   close   2323 non-null   float64       
 5   volume  2323 non-null   float64       
 6   date    2323 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 145.2+ KB


In [15]:
DataFrame.drop_duplicates(inplace=True)


In [16]:
DataFrame.reset_index(inplace=True)

In [17]:
DataFrame.drop('index',axis=1,inplace=True)

In [18]:
z = DataFrame[['date','open','high','close','low','volume','time']]

In [19]:
z

,date,open,high,close,low,volume,time
0,2016-01-02,435.67,437.56,435.40,432.41,3276.709621,1451692800
1,2016-01-03,435.40,435.75,431.91,425.02,3904.335318,1451779200
2,2016-01-04,431.90,435.79,433.85,431.37,5894.445723,1451865600
3,2016-01-05,433.84,435.64,433.34,430.00,5150.109476,1451952000
4,2016-01-06,433.32,433.46,430.87,428.15,5476.959959,1452038400
...,...,...,...,...,...,...,...
2317,2022-05-07,35999.10,36125.00,35458.00,34785.00,11372.453192,1651881600
2318,2022-05-08,35458.00,35499.93,34027.91,33701.00,22357.511521,1651968000
2319,2022-05-09,34033.49,34231.54,30078.27,30010.00,55136.453328,1652054400
2320,2022-05-10,30078.27,32645.49,31002.39,29735.05,55818.019003,1652140800


In [20]:
bb_indicator = BollingerBands(close = DataFrame['close'],window=20,window_dev=2,fillna = True)

In [21]:
z['bb_bbm'] = bb_indicator.bollinger_mavg()
z['bb_bbh'] = bb_indicator.bollinger_hband()
z['bb_bbl'] = bb_indicator.bollinger_lband()

C:\Users\shayanTH\AppData\Local\Temp\ipykernel_11912\3998541396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['bb_bbm'] = bb_indicator.bollinger_mavg()
C:\Users\shayanTH\AppData\Local\Temp\ipykernel_11912\3998541396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['bb_bbh'] = bb_indicator.bollinger_hband()
C:\Users\shayanTH\AppData\Local\Temp\ipykernel_11912\3998541396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [22]:
DataFrame = z

In [23]:
DataFrame.to_csv('Prices.csv')